In [18]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [19]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [20]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>Atrial Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [ ]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/hebbian_unsup/').glob('unet3d_swta_t')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(summary)

In [46]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_unet3d_swta_t')),
    #'H-EM-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_em_unet3d_swta_t')),
}

In [47]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

More than 1 tensorboard folder in run: runs/Atrial/semi_sup/h_unet3d_swta_t/inv_temp-1/regime-1/run-0


In [48]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

KeyError: 'segm/95hd'

<h2>Evaluation - Weight init comparison</h2>

<p>Evaluate weight init methods (only for baseline)</p>

In [43]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'Kaiming-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    'Xavier-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('xavier_unet3d')),
    'Orthogonal-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('orthogonal_unet3d')),
}

In [44]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [ ]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    #'VNet': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_vnet')),
    #'EM-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_em_unet3d')),
    #'CPS-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_cps_unet3d')),
    #'UAMT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_uamt_unet3d')),
    #'URPC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_urpc_unet3d')),
    #'CCT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_cct_unet3d')),
    #'DTC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_dtc_unet3d')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

<h1>LiTS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>Evaluation - Weight init comparison</h2>

<p>Evaluate weight init methods (only for baseline)</p>

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [36]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_unet3d')),
    #'VNet': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_vnet')),
    #'EM-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_em_unet3d')),
    #'CPS-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_cps_unet3d')),
    #'UAMT-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_uamt_unet3d')),
    #'URPC-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_urpc_unet3d')),
    #'CCT-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_cct_unet3d')),
    #'DTC-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_dtc_unet3d')),
}

In [37]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

Skipping not found: runs/LiTS/semi_sup/kaiming_unet3d/inv_temp-1/regime-1/run-0/test.csv
Skipping not found: runs/LiTS/semi_sup/kaiming_unet3d/inv_temp-1/regime-10/run-0/test.csv
Skipping not found: runs/LiTS/semi_sup/kaiming_unet3d/inv_temp-1/regime-10/run-1/test.csv


In [38]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet3D,0,1,2,66.27,57.35,89.85,25.17
1,UNet3D,0,1,5,68.49,59.53,80.43,23.02
2,UNet3D,1,1,2,62.13,54.37,84.33,24.52
3,UNet3D,1,1,5,66.79,58.10,79.40,22.52
4,UNet3D,2,1,2,59.88,52.67,95.97,28.99
5,UNet3D,2,1,5,67.40,58.76,98.97,27.61
6,UNet3D,3,1,2,58.83,52.99,78.01,25.24
7,UNet3D,3,1,5,65.38,56.77,87.70,24.10
8,UNet3D,4,1,2,66.81,58.70,83.74,23.17
9,UNet3D,4,1,5,64.83,57.00,82.17,26.13


Dice           Jaccard           Hausdorff Distance  \
                          Mean   CI 0.9%    Mean   CI 0.9%               Mean   
Model  Inv Temp Regime                                                          
UNet3D 1        2       62.784  3.099794  55.216  2.289930             86.380   
                5       66.578  1.269627  58.032  0.994116             85.734   

                                 Average Surface Distance            
                         CI 0.9%                     Mean   CI 0.9%  
Model  Inv Temp Regime                                               
UNet3D 1        2       5.802979                   25.418  1.844375  
                5       6.875144                   24.676  1.831152